In [1]:
import configparser
import os
from joblib import dump, load
import json
from tqdm import tqdm
from helpers.helper_functions import *
from helpers.helper_classes import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels as sm
import numpy as np
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA

pd.set_option('display.max_rows', 200)

# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')
# os.chdir(config['PATH']['ROOT_DIR'])

# # Load data
df = pd.read_csv(config['PATH']['DATA_DIR'] + '/dataset_mood_smartphone.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

# time to datetime
df['time'] = pd.to_datetime(df['time'])


In [2]:
# TODO: 
# DONE: Forward fill valence and arousal
# DONE: Remove appCat.builtin negative values
# DONE: Remove appCat outliers
# Impute valence, arousal and mood
# Impute long term missing values with mean instead of ffill (for valence and arousal)
# Aggregate to daily mood
# Decide on aggregation method for each variable

## Extreme values appCat variables
There are many outliers in the appCat variables, this is not ideal for numerical stability. We will one hot encode these outliers per variable and remove the outlier from the original observation

In [3]:
#  We will one hot encode these outliers per variable and remove the outlier from the original observation
#  Moreover we will remove all negative values

all_vars = df['variable'].unique()
appVars = [var for var in all_vars if 'appCat' in var]
appVars

for var in appVars:
    df_var_cur = df[df['variable'] == var]
    # Iterate over observations

    # Get 95th percentile
    perc98 = np.percentile(df_var_cur['value'], 98)

    # Get all idx where value is smaller than 0
    idx = df_var_cur[df_var_cur['value'] < 0].index
    df.drop(idx, axis=0, inplace=True)

    # Get all idx where value is larger than 95th percentile
    idx_98 = df_var_cur[df_var_cur['value'] > perc98].index

    # Change variable name to var_outlier
    df.loc[idx_98, 'variable'] = var + '_outlier'
    df.loc[idx_98, 'value'] = 1    


In [4]:
def KF_params(arma_model):
    p, _, q = arma_model.order
    params = arma_model.params()
    
    # if p == 0 and q == 0:
    #     return None
    m = np.maximum(p, q+1)
    T = np.zeros((m, m))
    R = np.zeros(m)
    a_init = np.zeros((m, 1))
    P_init = np.eye(m) * 10e7
    R[0] = 1
    Z = np.zeros(m)
    Z[0] = 1
    d = arma_model.params()['intercept'].astype(float)
    Q = np.array([params['sigma2']]).reshape(1,1)

    for i in range(p):
        T[i, 0] = params["ar.L"+str(i+1)]
    for j in range(q):
        R[j+1] = params["ma.L"+str(j+1)]
    if m > 1:
        for k in range(m-1):
            T[k, k+1] = 1
    elif p == 0 and q == 0:
        T[0,0] = 1
    
    R = R.reshape(m,1)
    Z = Z.reshape(1,m)

    # Sum AR coefficients
    sum_AR = np.sum(arma_model.arparams())
    sum_AR2 = np.sum(arma_model.arparams()**2)
    sum_MA2 = np.sum(arma_model.maparams()**2)

    # Initial state and variance
    if p > 0:
        mu = params['intercept'] / (1 - sum_AR)
        d = mu
        a_init[0] = mu
        if p > 1:
            for i in range(p-1):
                a_init[i+1] = np.sum(arma_model.arparams()[i+1] * mu)
        P_init[0,0] = params['sigma2'] * (sum_MA2 + 1) / (1 - sum_AR2)

    return {'T': T, 'R': R, 'Z': Z, 'Q': Q, 'd': d, 'a_init': a_init, 'P_init': P_init}

KF_res_dict = {}

# Iterate over people
# for person in tqdm(df['id'].unique()):
for person in ['AS14.28']:
    # Dataframe for mood of current person
    idx_mood = np.logical_and(df['id'] == person , df['variable'] == 'mood')
    df_mood = df[idx_mood].copy()['value']
    model = pm.auto_arima(df_mood, suppress_warnings=True, seasonal=False, stepwise=True, d = 0, stationary = True)

    # # Extract the best (p, d, q) orders
    p, d, q = model.order
    print(f"person: {person}, order: {model.order}")
    print(model.summary())
    print(model.arparams())
    params = model.params()

    # Get KF parameters
    KF_res_dict[person] = KF_params(model)




person: AS14.28, order: (1, 0, 0)
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  169
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -249.124
Date:                Thu, 13 Apr 2023   AIC                            504.248
Time:                        23:35:05   BIC                            513.638
Sample:                             0   HQIC                           508.058
                                - 169                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      5.2754      0.572      9.216      0.000       4.154       6.397
ar.L1          0.2574      0.082      3.146      0.002       0.097       0.418
sigma2         1.1

In [5]:
# # Load data
df = pd.read_csv(config['PATH']['DATA_DIR'] + '/dataset_mood_smartphone.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

# time to datetime
df['time'] = pd.to_datetime(df['time'])

for person in tqdm(df['id'].unique()):
    idx_mood = np.logical_and(df['id'] == person , df['variable'] == 'mood')
    df_mood = df[idx_mood].copy()
    df_mood

    # Get first date
    first_date = df_mood['time'].min().date()

    # Last date
    last_date = df_mood['time'].max().date()

    # Iterate over dates by day
    for date in pd.date_range(start=first_date, end=last_date, freq='D'):
        # Get all rows for this date
        idx_date = df_mood['time'].dt.date == date.date()
        df_date = df_mood[idx_date].copy()

        if len(df_date) == 5:
            continue

        # check for observation between 9.00 and 12.00
        hour_sets = [[9,12], [12,15], [15,18], [18,21], [21,24]]

        for hour_set in hour_sets:
            idx_cur = np.logical_and(df_date['time'].dt.hour >= hour_set[0], df_date['time'].dt.hour < hour_set[1])
            if len(df_date[idx_cur]) == 0:
                # Set hour of date
                cur_date = date.replace(hour=hour_set[0])
                # Create new row
                new_row = pd.DataFrame({'id': person, 'time': cur_date, 'variable': 'mood', 'value': np.nan}, index=[0])
                df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)
                df_date = pd.concat([df_date, pd.DataFrame(new_row)], ignore_index=True)



100%|██████████| 27/27 [00:24<00:00,  1.10it/s]


In [6]:
# AS14.05
person_sel = "AS14.28"
idx_mood = np.logical_and(df['id'] == person_sel, df['variable'] == 'mood')
KF_1430 = KF_res_dict[person_sel]
df_mood = df[idx_mood].copy()
# sort by time
df_mood.sort_values('time', inplace=True)
# df_mood.values to floats
df_mood['value'] = df_mood['value'].astype(float)

KF = KalmanFilter(y=df_mood['value'].values,
                a_init=KF_1430['a_init'],
                P_init = KF_1430['P_init'],
                H = np.array([0]).reshape(1,1),
                Q = KF_1430['Q'],
                R = KF_1430['R'],
                d = KF_1430['d'])

res = KF.run_smoother(Z = KF_1430['Z'], T = KF_1430['T'])

res_final = (res['a_smooth'][:, 0] + KF_1430['d']).flatten()
# DF with one column with the smoothed values, and one column with the true values
df_mood['smoothed'] = res_final
df_mood

,id,time,variable,value,smoothed
377813,AS14.28,2014-03-31 09:00:00,mood,NaN,14.204316
377814,AS14.28,2014-03-31 12:00:00,mood,NaN,8.917793
377815,AS14.28,2014-03-31 15:00:00,mood,NaN,7.516804
4543,AS14.28,2014-03-31 18:00:00,mood,7.0,7.000000
4544,AS14.28,2014-03-31 21:00:00,mood,6.0,6.000000
377816,AS14.28,2014-04-01 09:00:00,mood,NaN,7.053827
4545,AS14.28,2014-04-01 12:00:00,mood,8.0,8.000000
4546,AS14.28,2014-04-01 15:00:00,mood,8.0,8.000000
4547,AS14.28,2014-04-01 18:00:00,mood,7.0,7.000000
4548,AS14.28,2014-04-01 21:00:00,mood,7.0,7.000000


In [114]:
KF_1430

{'T': array([[0.25740686]]),
 'R': array([[1.]]),
 'Z': array([[1.]]),
 'Q': array([[1.11618766]]),
 'd': 7.1040789058199625,
 'a_init': array([[7.10407891]]),
 'P_init': array([[1.19539231]])}

In [49]:
# first column of res['a_smooth']
res['a_smooth'][:,0] 

array([7.42064804, 6.42064804, 5.42064804, 6.42064804, 5.42064804,
       4.42064804, 5.42064804, 6.42064804, 5.42064804, 4.42064804,
       3.42064804, 5.42064804, 7.42064804, 6.42064804, 6.42064804,
       6.42064804, 7.42064804, 7.42064804, 6.42064804, 6.42064804,
       5.42064804, 6.42064804, 6.42064804, 7.42064804, 5.42064804,
       6.42064804, 7.42064804, 7.42064804, 6.42064804, 6.42064804,
       6.42064804, 3.42064804, 2.42064804, 3.42064804, 3.42064804,
       2.42064804, 5.42064804, 3.42064804, 3.42064804, 4.42064804,
       2.42064804, 3.42064804, 3.42064804, 3.42064804, 4.42064804,
       4.42064804, 5.42064804, 4.42064804, 5.42064804, 5.42064804,
       4.42064804, 4.42064804, 5.42064804, 5.42064804, 5.42064804,
       5.42064804, 5.42064804, 5.42064804, 5.42064804, 6.42064804,
       5.42064804, 6.42064804, 5.42064804, 6.42064804, 5.42064804,
       5.42064804, 5.42064804, 5.42064804, 5.42064804, 6.42064804,
       5.42064804, 6.42064804, 6.42064804, 6.42064804, 5.42064

In [33]:
Z = KF_1430['Z']
P = KF_1430['P_init']
a_init = KF_1430['a_init']
T = KF_1430['T']
H = np.array([0]).reshape(1,1)

T @ P @ T.T

array([[1.00000003e+08, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00]])

In [10]:
KF_res_dict[person_sel]

{'T': array([[0.91391456, 1.        ],
        [0.        , 0.        ]]),
 'R': array([[ 1.        ],
        [-0.55195327]]),
 'Z': array([[1., 0.]]),
 'Q': array([0.52317]),
 'd': 0.5793519640486993,
 'a_init': array([6.7299648, 0.       ]),
 'P_init': array([[4.14271841e+00, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e+08]])}

In [49]:
# Get all variables
variables = df['variable'].unique()

# Iterate over variables
for variable in variables:
    # Get the data for this variable
    df_variable = df[df['variable'] == variable]

    # Get 95th percentile
    percentile_95 = np.percentile(df_variable['value'], 95)

    # Describe value column for variable
    print(f"Variable {variable}:")
    print(df_variable['value'].describe())

Variable mood:
count    5641.000000
mean        6.992555
std         1.032769
min         1.000000
25%         7.000000
50%         7.000000
75%         8.000000
max        10.000000
Name: value, dtype: float64
Variable circumplex.arousal:
count    5597.000000
mean       -0.098624
std         1.051868
min        -2.000000
25%        -1.000000
50%         0.000000
75%         1.000000
max         2.000000
Name: value, dtype: float64
Variable circumplex.valence:
count    5487.000000
mean        0.687808
std         0.671298
min        -2.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: value, dtype: float64
Variable activity:
count    22965.000000
mean         0.115958
std          0.186946
min          0.000000
25%          0.000000
50%          0.021739
75%          0.158333
max          1.000000
Name: value, dtype: float64
Variable screen:
count    96578.000000
mean        75.335206
std        253.822497
min          0.035000
25%        

 56%|█████▌    | 15/27 [00:12<00:09,  1.23it/s]


KeyboardInterrupt: 